# Day 10: Monitoring Station
https://adventofcode.com/2019/day/10

## Part 1

In [ ]:
import numpy as np
import urllib.request
import math
import itertools
import re

In [ ]:
def checkMapIsValid(string_map):
    #Debe haber filas en el mapa
    if len(string_map) == 0:
        return False
    #La primera fila debe tener columnas
    width = len(string_map[0])
    if width == 0:
        return False
    
    #Todas las filas deben tener la misma longitud
    for num_line in range(1, len(string_map)):
        if len(string_map[num_line]) != width:
            return False
    return True
    
def loadMap(string_map):
    if not checkMapIsValid(string_map):
        print('El mapa no es válido.')
        return None
    
    height = len(string_map)
    width  = len(string_map[0])
    
    int_map = np.zeros([height, width])
    for num_line in range(height):
        line = string_map[num_line]
        for num_col in range(width):
            asteroid = line[num_col:num_col+1]
            if asteroid == '#':
                int_map[num_line][num_col] = 1
    
    return int_map

In [ ]:
string_map = ['..###.',
              '.#....']

In [ ]:
string_map

In [ ]:
int_map = loadMap(string_map)
int_map

In [ ]:
def probeAsteroid(int_map, asteroid_col, asteroid_line):
    
    height = len(int_map)
    width  = len(int_map[0])
    
    # Comprobar que soy un asteroide...
    assert int_map[asteroid_line][asteroid_col] == 1, f'There is no asteroid on [{asteroid_col}, {asteroid_line}]!'
    
    # obtener los vectores a cada uno de los demás asteroides
    dict_vectores = {}
    for line in range(height):
        for col in range(width):
            #Un asteroide no se detecta a sí mismo
            if line == asteroid_line and col == asteroid_col:
                continue
            if int_map[line][col] == 0:
                continue
            norm_line = line - asteroid_line
            norm_col  = col - asteroid_col
            modulus = math.sqrt(math.pow(norm_line, 2) + math.pow(norm_col, 2))
            norm_line = round(norm_line / modulus, 10)
            norm_col  = round(norm_col / modulus, 10)
            norm_coords = ( norm_col, norm_line )
            
            if not norm_coords in dict_vectores:
                dict_vectores[norm_coords] = []
            dict_vectores[norm_coords].append( ( modulus, ( col, line )  ) )

    print(dict_vectores)
    #Para cada dirección, nos quedamos con el que menos módulo tiene
    dict_visibles = {}
    dict_novisibles = {}
    for norm_coords in dict_vectores:
        print(norm_coords)
        candidates = dict_vectores[norm_coords]
        candidates.sort()
        print(candidates)
        print(20 * '-')
        (modulus, map_coords) = candidates[0]
        dict_visibles[map_coords] = modulus
        for otro_asteroide in candidates[1:]:
            (modulus, map_coords) = otro_asteroide
            dict_novisibles[map_coords] = candidates[0]
            
    return len(dict_visibles), dict_vectores, dict_visibles, dict_novisibles

In [ ]:
num_visibles, dic, dic_vis, dic_novis = probeAsteroid(int_map, 2, 0)

In [ ]:
dic

In [ ]:
dic_vis

In [ ]:
dic_novis

In [ ]:
def searchBestLocation(int_map):
    height = len(int_map)
    width  = len(int_map[0])
    
    best_bet = -1
    best_asteroid = None

    for line in range(height):
        for col in range(width):
            if int_map[line][col] == 0:
                continue
            num_asteroids, _, _, _ = probeAsteroid(int_map, col, line)
            
            if num_asteroids > best_bet:
                best_bet = num_asteroids
                best_asteroid = (col, line)
                
    return best_asteroid, best_bet

In [ ]:
searchBestLocation(int_map)

### Test

#### Test 1:

Map:

```
.#..#
.....
#####
....#
...##
```

Best location is (3, 4), 8 visible asteroids

In [ ]:
string_map = ['.#..#',
            '.....',
            '#####',
            '....#',
            '...##']

int_map = loadMap(string_map)
print(int_map)
searchBestLocation(int_map)

In [ ]:
probeAsteroid(int_map, 1, 0)

In [ ]:
probeAsteroid(int_map, 3, 4)

#### Test 2:

Map:

```
......#.#.
#..#.#....
..#######.
.#.#.###..
.#..#.....
..#....#.#
#..#....#.
.##.#..###
##...#..#.
.#....####
```

Best location is (5, 8), 33 visible asteroids

In [ ]:
string_map = ['......#.#.',
'#..#.#....',
'..#######.',
'.#.#.###..',
'.#..#.....',
'..#....#.#',
'#..#....#.',
'.##.#..###',
'##...#..#.',
'.#....####']


int_map = loadMap(string_map)
print(int_map)
searchBestLocation(int_map)

#### Test 3:

Map:

```
#.#...#.#.
.###....#.
.#....#...
##.#.#.#.#
....#.#.#.
.##..###.#
..#...##..
..##....##
......#...
.####.###.
```

Best location is (1, 2), 35 visible asteroids

In [ ]:
string_map = ['#.#...#.#.',
'.###....#.',
'.#....#...',
'##.#.#.#.#',
'....#.#.#.',
'.##..###.#',
'..#...##..',
'..##....##',
'......#...',
'.####.###.']

int_map = loadMap(string_map)
print(int_map)
searchBestLocation(int_map)

#### Test 4:

Map:

```
.#..#..###
####.###.#
....###.#.
..###.##.#
##.##.#.#.
....###..#
..#.#..#.#
#..#.#.###
.##...##.#
.....#.#..
```

Best location is (6, 3), 41 visible asteroids

In [ ]:
string_map = ['.#..#..###',
'####.###.#',
'....###.#.',
'..###.##.#',
'##.##.#.#.',
'....###..#',
'..#.#..#.#',
'#..#.#.###',
'.##...##.#',
'.....#.#..']


int_map = loadMap(string_map)
print(int_map)
searchBestLocation(int_map)

#### Test 5:

Map:

```
.#..##.###...#######
##.############..##.
.#.######.########.#
.###.#######.####.#.
#####.##.#.##.###.##
..#####..#.#########
####################
#.####....###.#.#.##
##.#################
#####.##.###..####..
..######..##.#######
####.##.####...##..#
.#####..#.######.###
##...#.##########...
#.##########.#######
.####.#.###.###.#.##
....##.##.###..#####
.#.#.###########.###
#.#.#.#####.####.###
###.##.####.##.#..##
```

Best location is (11, 13), 210 visible asteroids

In [ ]:
string_map = ['.#..##.###...#######',
'##.############..##.',
'.#.######.########.#',
'.###.#######.####.#.',
'#####.##.#.##.###.##',
'..#####..#.#########',
'####################',
'#.####....###.#.#.##',
'##.#################',
'#####.##.###..####..',
'..######..##.#######',
'####.##.####...##..#',
'.#####..#.######.###',
'##...#.##########...',
'#.##########.#######',
'.####.#.###.###.#.##',
'....##.##.###..#####',
'.#.#.###########.###',
'#.#.#.#####.####.###',
'###.##.####.##.#..##']


int_map = loadMap(string_map)
print(int_map)
searchBestLocation(int_map)

### Solution

In [ ]:
input_10 = r'aoc2019-input-day10.txt'
with open(input_10, 'r') as f:
#     data9 = [int(data) for data in f.read().split(',') if len(data) > 0]
    data10 = [re.sub("\n", "", l) for l in f.readlines()]
data10

In [ ]:
int_map = loadMap(data10)
print(int_map)
searchBestLocation(int_map)

>>>SOLUTION: 253

## Part 2

In [ ]:
num_visibles, dic, dic_vis, dic_novis = probeAsteroid(int_map, 11, 19)

In [ ]:
num_visibles

In [ ]:
dic